<a href="https://colab.research.google.com/github/ganeshchakravarthy/projects/blob/main/Fine_tune_Bert_Music_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets torch scikit-learn pandas

In [2]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import files
uploaded = files.upload()

Saving music_movies_dataset.csv to music_movies_dataset.csv


In [4]:
df = pd.read_csv("music_movies_dataset.csv")

In [5]:
print(df.head())

                                    text  label
0                  Play some jazz music.      0
1                Play Bohemian Rhapsody.      0
2      Alexa, play my favorite playlist.      0
3     Can you play relaxing piano music?      0
4  Play the latest song by Taylor Swift.      0


In [7]:
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 15
})


In [8]:
from transformers import AutoTokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
from sklearn.model_selection import train_test_split
df = tokenized_datasets.to_pandas()

train_texts, val_texts, train_labels, val_labels = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

train_data = Dataset.from_pandas(pd.DataFrame({"text": train_texts, "label": train_labels}))
val_data = Dataset.from_pandas(pd.DataFrame({"text": val_texts, "label": val_labels}))

print(f"Training Samples: {len(train_data)}")
print(f"Validation Samples: {len(val_data)}")

Training Samples: 12
Validation Samples: 3


In [27]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [28]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
training_args = TrainingArguments(
    output_dir="./bert_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2,
    logging_steps=1,
)

In [30]:
# ✅ Step 8: Define Trainer & Start Training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.658700,0.631617
2,0.670300,0.643409
3,0.600100,0.638575


TrainOutput(global_step=9, training_loss=0.6668732696109347, metrics={'train_runtime': 291.3573, 'train_samples_per_second': 0.124, 'train_steps_per_second': 0.031, 'total_flos': 9471997992960.0, 'train_loss': 0.6668732696109347, 'epoch': 3.0})

In [32]:
print(trainer.evaluate())


{'eval_loss': 0.6385751962661743, 'eval_runtime': 6.9446, 'eval_samples_per_second': 0.432, 'eval_steps_per_second': 0.144, 'epoch': 3.0}


In [34]:
import torch
def predict_text(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits).item()
    return predicted_class

test_sentences = ["Play the latest pop song",
                  "I want to watch Avengers: Endgame",
                  "Can you play an acoustic guitar playlist?",
                  "Find and stream a sci-fi movie",
                  "Play some relaxing instrumental music"
                  ]

for sentence in test_sentences:
    predicted_class = predict_text(sentence)
    category = "Music" if predicted_class ==0 else "Movies"
    print(f"Sentence: '{sentence}' | Predicted Category: {category}")

Sentence: 'Play the latest pop song' | Predicted Category: Movies
Sentence: 'I want to watch Avengers: Endgame' | Predicted Category: Movies
Sentence: 'Can you play an acoustic guitar playlist?' | Predicted Category: Movies
Sentence: 'Find and stream a sci-fi movie' | Predicted Category: Movies
Sentence: 'Play some relaxing instrumental music' | Predicted Category: Movies
